In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Wed Mar 27 08:34:53 PDT 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
roadrunner

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                16
On-line CPU(s) list:   0-15
Thread(s) per core:    2
Core(s) per socket:    4
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 26
Model name:            Intel(R) Xeon(R) CPU           E5520  @ 2.27GHz
Stepping:              5
CPU MHz:               1596.000
CPU max MHz:           2394.0000
CPU min MHz:           1596.0000
BogoMIPS:              4521.81
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              8192K
NUMA node0 CPU(s):     0-15
Flags:                 fpu vme de pse tsc msr pae m

No LSB modules are available.


In [2]:
%env wd=/home/sam/analyses/20190327_cgig_repeatmasker_all
%env fasta_url=http://owl.fish.washington.edu/halfshell/genomic-databank/Crassostrea_gigas.oyster_v9.dna_sm.toplevel.fa
%env fasta=Crassostrea_gigas.oyster_v9.dna_sm.toplevel.fa
%env repeat_masker=/home/shared/RepeatMasker-4.0.7/RepeatMasker
%env cpus=16
%env checksum=6de9d1239eb10ea0545bed6c4e746d6c

env: wd=/home/sam/analyses/20190327_cgig_repeatmasker_all
env: fasta_url=http://owl.fish.washington.edu/halfshell/genomic-databank/Crassostrea_gigas.oyster_v9.dna_sm.toplevel.fa
env: fasta=Crassostrea_gigas.oyster_v9.dna_sm.toplevel.fa
env: repeat_masker=/home/shared/RepeatMasker-4.0.7/RepeatMasker
env: cpus=16
env: checksum=6de9d1239eb10ea0545bed6c4e746d6c


#### Create necessary directories

In [3]:
%%bash
mkdir --parents ${wd}

#### Download _Crassostrea gigas genome FastA file

Info on FastA file is here: [https://github.com/RobertsLab/resources/wiki/Genomic-Resources#genome](https://github.com/RobertsLab/resources/wiki/Genomic-Resources#genome)

In [4]:
%%bash
# Using rsync
cd ${wd}

rsync \
--archive \
--verbose \
--progress \
owl:/volume1/web/halfshell/genomic-databank/Crassostrea_gigas.oyster_v9.dna_sm.toplevel.fa .

ls -lh

receiving incremental file list
Crassostrea_gigas.oyster_v9.dna_sm.toplevel.fa
    567,592,991 100%    5.15MB/s    0:01:45 (xfr#1, to-chk=0/1)

sent 30 bytes  received 567,662,409 bytes  5,137,216.64 bytes/sec
total size is 567,592,991  speedup is 1.00
total 542M
-rw-rw-rw- 1 sam users 542M Aug 24  2018 Crassostrea_gigas.oyster_v9.dna_sm.toplevel.fa


#### If need to download via wget, change cell below to code, instead of markdown

#### Also, change cell with md5 checksum comparisons to code, instead of markdown

%%bash
time \
wget ${fasta_url} \
--quiet \
--directory-prefix=${wd}

ls -lh ${wd}

#### Verify MD5 checksum

Original MD5 checksum taken from GitHub Genomic Resource linked above.

Use ```md5sum``` to generate checksum from downloaded FastA file and ```awk``` to print the first field (i.e. the checksum value). This is saved to the variable: ```dl_md5```

Then, check for differences between the two variables. 

No output confirms no difference.

%%bash
md5=${checksum}
dl_md5=$(md5sum ${wd}/${fasta} | awk '{ print $1 }')
diff <(echo "$md5") <(echo "$dl_md5")

#### Run RepeatMasker with _all_ species setting and following options:

```-species "all"``` : Sets species to all

```-par 15``` : Use 15 CPU threads

```-gff``` : Create GFF output file (in addition to default files)

```-excln``` : Adjusts output table calculations to exclude sequence runs of >=25Ns. Useful for draft genome assemblies.

```-1>``` : Send stdout to file instead of printing to notebook.

```-2>``` : Send stderr to file instead of printing to notebook.


In [ ]:
%%bash

cd ${wd}
time \
${repeat_masker} \
${fasta} \
-species "all" \
-par ${cpus} \
-gff \
-excln \
1> stdout.out \
2> stderr.err

sed '/^Subject:/ s/ / repeatmasker_gigas_all JOB COMPLETE/' ~/.default-subject.mail | msmtp "$EMAIL"

In [6]:
%%bash
ls /home/sam/data/Cvirginica_genome/

Cvirginica_v300.fa


In [7]:
%%bash
ls /home/sam/analyses/20180822_virginica_repeatmasker_all/

Cvirginica_v300.fa.cat.gz
Cvirginica_v300.fa.masked
Cvirginica_v300.fa.out
Cvirginica_v300.fa.out.gff
Cvirginica_v300.fa.tbl
stderr.err
stdout.out


#### Run RepeatMasker with _crassostrea gigas_ species setting and following options:

```-species "crassostrea gigas"``` : Sets species to _Crassostrea gigas_

```-par 15``` : Use 15 CPU threads

```-gff``` : Create GFF output file (in addition to default files)

```-excln``` : Adjusts output table calculations to exclude sequence runs of >=25Ns. Useful for draft genome assemblies.

```-1>``` : Send stdout to file instead of printing to notebook.

```-2>``` : Send stderr to file instead of printing to notebook.


In [8]:
%%bash
wd="/home/sam/analyses/20180822_virginica_repeatmasker_Cgigas"
cd /home/sam/data/Cvirginica_genome/
time \
/home/shared/RepeatMasker-4.0.7/RepeatMasker \
/home/sam/data/Cvirginica_genome/Cvirginica_v300.fa \
-species "crassostrea gigas" \
-par 15 \
-gff \
-excln \
1> "$wd"/stdout.out \
2> "$wd"/stderr.err
find ./ -not -name Cvirginica_v300.fa -exec mv '{}' "$wd" \;
sed '/^Subject:/ s/ / repeatmasker_virginica_gigas JOB COMPLETE/' ~/.default-subject.mail | msmtp "$EMAIL"


real	161m24.668s
user	1801m15.348s
sys	125m13.016s
mv: cannot move './' to '/home/sam/analyses/20180822_virginica_repeatmasker_Cgigas/.': Device or resource busy


In [9]:
%%bash
ls /home/sam/data/Cvirginica_genome/

Cvirginica_v300.fa


In [10]:
%%bash
ls /home/sam/analyses/20180822_virginica_repeatmasker_Cgigas

Cvirginica_v300.fa.cat.gz
Cvirginica_v300.fa.masked
Cvirginica_v300.fa.out
Cvirginica_v300.fa.out.gff
Cvirginica_v300.fa.tbl
stderr.err
stdout.out


#### Run RepeatMasker with _crassostrea virginica_ species setting and following options:

```-species "crassostrea virginica"``` : Sets species to _Crassostrea virginica_

```-par 15``` : Use 15 CPU threads

```-gff``` : Create GFF output file (in addition to default files)

```-excln``` : Adjusts output table calculations to exclude sequence runs of >=25Ns. Useful for draft genome assemblies.

```-1>``` : Send stdout to file instead of printing to notebook.

```-2>``` : Send stderr to file instead of printing to notebook.

In [11]:
%%bash
wd="/home/sam/analyses/20180822_virginica_repeatmasker_Cvirginica"
cd /home/sam/data/Cvirginica_genome/
time \
/home/shared/RepeatMasker-4.0.7/RepeatMasker \
/home/sam/data/Cvirginica_genome/Cvirginica_v300.fa \
-species "crassostrea virginica" \
-par 15 \
-gff \
-excln \
1> "$wd"/stdout.out \
2> "$wd"/stderr.err
find ./ -not -name Cvirginica_v300.fa -exec mv '{}' "$wd" \;
sed '/^Subject:/ s/ / repeatmasker_virginica_virginica JOB COMPLETE/' ~/.default-subject.mail | msmtp "$EMAIL"


real	88m45.684s
user	911m23.604s
sys	139m47.128s
mv: cannot move './' to '/home/sam/analyses/20180822_virginica_repeatmasker_Cvirginica/.': Device or resource busy


In [12]:
%%bash
ls /home/sam/data/Cvirginica_genome/

Cvirginica_v300.fa


In [13]:
%%bash
ls /home/sam/analyses/20180822_virginica_repeatmasker_Cvirginica

Cvirginica_v300.fa.cat.gz
Cvirginica_v300.fa.masked
Cvirginica_v300.fa.out
Cvirginica_v300.fa.out.gff
Cvirginica_v300.fa.tbl
stderr.err
stdout.out


#### Run RepeatMasker with _defaults_ species setting and following options:

```-par 15``` : Use 15 CPU threads

```-gff``` : Create GFF output file (in addition to default files)

```-excln``` : Adjusts output table calculations to exclude sequence runs of >=25Ns. Useful for draft genome assemblies.

```-1>``` : Send stdout to file instead of printing to notebook.

```-2>``` : Send stderr to file instead of printing to notebook.

In [14]:
%%bash
wd="/home/sam/analyses/20180822_virginica_repeatmasker_defaults"
cd /home/sam/data/Cvirginica_genome/
time \
/home/shared/RepeatMasker-4.0.7/RepeatMasker \
/home/sam/data/Cvirginica_genome/Cvirginica_v300.fa \
-par 15 \
-gff \
-excln \
1> "$wd"/stdout.out \
2> "$wd"/stderr.err
find ./ -not -name Cvirginica_v300.fa -exec mv '{}' "$wd" \;
sed '/^Subject:/ s/ / repeatmasker_virginica_defaults JOB COMPLETE/' ~/.default-subject.mail | msmtp "$EMAIL"


real	148m33.929s
user	1883m54.940s
sys	175m28.320s
mv: cannot move './' to '/home/sam/analyses/20180822_virginica_repeatmasker_defaults/.': Device or resource busy


In [15]:
%%bash
ls /home/sam/data/Cvirginica_genome/

Cvirginica_v300.fa


In [16]:
%%bash
ls /home/sam/analyses/20180822_virginica_repeatmasker_defaults

Cvirginica_v300.fa.cat.gz
Cvirginica_v300.fa.masked
Cvirginica_v300.fa.out
Cvirginica_v300.fa.out.gff
Cvirginica_v300.fa.tbl
stderr.err
stdout.out


#### SUMMARY TABLE (species=all)

In [17]:
%%bash
cat /home/sam/analyses/20180822_virginica_repeatmasker_all/Cvirginica_v300.fa.tbl

file name: Cvirginica_v300.fa       
sequences:            11
total length:  684741128 bp  (684675328 bp excl N/X-runs)
GC level:         34.83 %
bases masked:  113771462 bp ( 16.62 %)
               number of      length   percentage
               elements*    occupied  of sequence
--------------------------------------------------
Retroelements        97003     27946871 bp    4.08 %
   SINEs:            48145      9242559 bp    1.35 %
   Penelope           1429       256929 bp    0.04 %
   LINEs:            27022     10570154 bp    1.54 %
    CRE/SLACS           28         2219 bp    0.00 %
     L2/CR1/Rex       2160       316660 bp    0.05 %
     R1/LOA/Jockey    3058       386611 bp    0.06 %
     R2/R4/NeSL        511       226938 bp    0.03 %
     RTE/Bov-B        7377      3276312 bp    0.48 %
     L1/CIN4          1331        95476 bp    0.01 %
   LTR elements:     21836      8134158 bp    1.19 %
     BEL/Pao          1807       936488 bp    0.14 %
     Ty1/Copia        3046  

#### SUMMARY TABLE (species=_Crassostrea gigas_)

In [18]:
%%bash
cat /home/sam/analyses/20180822_virginica_repeatmasker_Cgigas/Cvirginica_v300.fa.tbl

file name: Cvirginica_v300.fa       
sequences:            11
total length:  684741128 bp  (684675328 bp excl N/X-runs)
GC level:         34.83 %
bases masked:   93923386 bp ( 13.72 %)
               number of      length   percentage
               elements*    occupied  of sequence
--------------------------------------------------
Retroelements        26397     15008601 bp    2.19 %
   SINEs:                4          722 bp    0.00 %
   Penelope            675       190160 bp    0.03 %
   LINEs:            17645      8922188 bp    1.30 %
    CRE/SLACS            0            0 bp    0.00 %
     L2/CR1/Rex         70        39188 bp    0.01 %
     R1/LOA/Jockey       0            0 bp    0.00 %
     R2/R4/NeSL          4         5110 bp    0.00 %
     RTE/Bov-B        6194      2718955 bp    0.40 %
     L1/CIN4             0            0 bp    0.00 %
   LTR elements:      8748      6085691 bp    0.89 %
     BEL/Pao           933       788887 bp    0.12 %
     Ty1/Copia          47  

#### SUMMARY TABLE (species=_Crassostrea virginica_)

In [19]:
%%bash
cat /home/sam/analyses/20180822_virginica_repeatmasker_Cvirginica/Cvirginica_v300.fa.tbl

file name: Cvirginica_v300.fa       
sequences:            11
total length:  684741128 bp  (684675328 bp excl N/X-runs)
GC level:         34.83 %
bases masked:   46637065 bp ( 6.81 %)
               number of      length   percentage
               elements*    occupied  of sequence
--------------------------------------------------
Retroelements        43139      8952068 bp    1.31 %
   SINEs:            43139      8952068 bp    1.31 %
   Penelope              0            0 bp    0.00 %
   LINEs:                0            0 bp    0.00 %
    CRE/SLACS            0            0 bp    0.00 %
     L2/CR1/Rex          0            0 bp    0.00 %
     R1/LOA/Jockey       0            0 bp    0.00 %
     R2/R4/NeSL          0            0 bp    0.00 %
     RTE/Bov-B           0            0 bp    0.00 %
     L1/CIN4             0            0 bp    0.00 %
   LTR elements:         0            0 bp    0.00 %
     BEL/Pao             0            0 bp    0.00 %
     Ty1/Copia           0   

#### SUMMARY TABLE (species=default)

In [20]:
%%bash
cat /home/sam/analyses/20180822_virginica_repeatmasker_defaults/Cvirginica_v300.fa.tbl

file name: Cvirginica_v300.fa       
sequences:            11
total length:  684741128 bp  (684675328 bp excl N/X-runs)
GC level:         34.83 %
bases masked:   13461422 bp ( 1.97 %)
               number of      length   percentage
               elements*    occupied  of sequence
--------------------------------------------------
SINEs:             2056       120820 bp    0.02 %
      ALUs            0            0 bp    0.00 %
      MIRs          240        14635 bp    0.00 %

LINEs:             3408       331585 bp    0.05 %
      LINE1         240        16835 bp    0.00 %
      LINE2         728        69177 bp    0.01 %
      L3/CR1       1369       135234 bp    0.02 %

LTR elements:       704       236625 bp    0.03 %
      ERVL           14          944 bp    0.00 %
      ERVL-MaLRs     12          892 bp    0.00 %
      ERV_classI    272        36695 bp    0.01 %
      ERV_classII     4          206 bp    0.00 %

DNA elements:      1088       100026 bp    0.01 %
     hAT-Cha

Files copied to owl/Athaliana outside of notebook due to ```sudo``` requirement.

In [21]:
%%bash
ls /mnt/owl/Athaliana/20180822_virginica_repeatmasker_*

/mnt/owl/Athaliana/20180822_virginica_repeatmasker_all:
Cvirginica_v300.fa.cat.gz
Cvirginica_v300.fa.masked
Cvirginica_v300.fa.out
Cvirginica_v300.fa.out.gff
Cvirginica_v300.fa.tbl
stderr.err
stdout.out

/mnt/owl/Athaliana/20180822_virginica_repeatmasker_Cgigas:
Cvirginica_v300.fa.cat.gz
Cvirginica_v300.fa.masked
Cvirginica_v300.fa.out
Cvirginica_v300.fa.out.gff
Cvirginica_v300.fa.tbl
stderr.err
stdout.out

/mnt/owl/Athaliana/20180822_virginica_repeatmasker_Cvirginica:
Cvirginica_v300.fa.cat.gz
Cvirginica_v300.fa.masked
Cvirginica_v300.fa.out
Cvirginica_v300.fa.out.gff
Cvirginica_v300.fa.tbl
stderr.err
stdout.out

/mnt/owl/Athaliana/20180822_virginica_repeatmasker_defaults:
Cvirginica_v300.fa.cat.gz
Cvirginica_v300.fa.masked
Cvirginica_v300.fa.out
Cvirginica_v300.fa.out.gff
Cvirginica_v300.fa.tbl
stderr.err
stdout.out
